# Titanic Dataset: Data Inspection and Missing Value Handling
**Author:** Apeksha Shenoy Mangalpady
**Purpose:** Load and inspect Titanic dataset, identify missing values, and prepare initial features.

In [5]:
import pandas as pd
import numpy as np


In [ ]:
train_data = pd.read_csv('D:/CAPSTONE_PROJECT/Titanic_Capstone_Project/Data/train.csv')
test_data = pd.read_csv('D:/CAPSTONE_PROJECT/Titanic_Capstone_Project/Data/test.csv')
gender_data = pd.read_csv('D:/CAPSTONE_PROJECT/Titanic_Capstone_Project/Data/gender_submission.csv')